# <center>Lab Sheet 9</center>

# <center>Neurodynamics and Hopfield Networks</center>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
np.random.seed(42)
os.makedirs("lab9_plots", exist_ok=True)

# Print the name and roll number
print("Name: Somesh Singh")
print("Roll Number: 233025921")

Name: Somesh Singh
Roll Number: 233025921


# -------------------------
# Utilities
# -------------------------

In [2]:
def bipolar(x):
    """Convert {0,1} to bipolar {-1,+1}"""
    x = np.array(x)
    return np.where(x==0, -1, 1)

def binarize_from_bipolar(x):
    """Convert bipolar (-1, +1) -> (0,1)"""
    return ((np.array(x) + 1) // 2).astype(int)

# Print the name and roll number
print("Name: Somesh Singh")
print("Roll Number: 233025921")

def plot_pattern(vec, shape, title, path):
    plt.figure(figsize=(2.5,2.5))
    plt.imshow(vec.reshape(shape), cmap='gray', vmin=-1, vmax=1)
    plt.title(title)
    plt.axis('off')
    plt.savefig(path, dpi=150, bbox_inches='tight')
    plt.close()

Name: Somesh Singh
Roll Number: 233025921


# -------------------------
# Hopfield: Hebbian training (zero diagonal)
# -------------------------

In [3]:
class Hopfield:
    def __init__(self, n):
        self.n = n
        self.W = np.zeros((n, n), dtype=float)

    def store_patterns(self, patterns):
        """patterns: array of shape (P, n) in bipolar (-1,+1)"""
        P = len(patterns)
        self.W[:] = 0.0
        for p in patterns:
            self.W += np.outer(p, p)
        # zero diagonal
        np.fill_diagonal(self.W, 0.0)
        # normalize by N optionally (classic Hopfield uses 1/N)
        self.W /= self.n

    def energy(self, state):
        """Energy E = -1/2 s^T W s"""
        s = state.reshape(self.n)
        return -0.5 * s @ (self.W @ s)

    def sign_update(self, h):
        # sign function with zero -> +1
        return np.where(h >= 0, 1, -1)

    def synchronous_update(self, state):
        h = self.W @ state
        return self.sign_update(h)

    def asynchronous_update(self, state, steps=1, random_order=True):
        s = state.copy()
        history = [s.copy()]
        for t in range(steps):
            indices = np.arange(self.n)
            if random_order:
                np.random.shuffle(indices)
            for i in indices:
                h_i = self.W[i, :] @ s
                s[i] = 1 if h_i >= 0 else -1
            history.append(s.copy())
        return s, history

# Print the name and roll number
print("Name: Somesh Singh")
print("Roll Number: 233025921")

Name: Somesh Singh
Roll Number: 233025921


# -------------------------
# Create patterns (simple 8x8 binary images)
# -------------------------

In [4]:
def create_base_patterns(shape=(8,8)):
    h, w = shape
    N = h*w
    patterns = []
    # Pattern A: diagonal line
    A = np.zeros((h,w), dtype=int); np.fill_diagonal(A, 1)
    patterns.append(bipolar(A.flatten()))
    # Print the name and roll number
    print("Name: Somesh Singh")
    print("Roll Number: 233025921")
    # Pattern B: cross
    B = np.zeros((h,w), dtype=int)
    B[h//2,:] = 1
    B[:,w//2] = 1
    patterns.append(bipolar(B.flatten()))
    # Pattern C: checkerboard
    C = np.indices((h,w)).sum(axis=0) % 2
    patterns.append(bipolar(C.flatten()))
    return np.array(patterns), (h,w)

**Q1. Implement a Hopfield network to store and recall binary patterns.**

**Q2. Simulate pattern distortion and recovery using asynchronous updates.**

In [5]:
patterns, shape = create_base_patterns((8,8))
N = patterns.shape[1]
hop = Hopfield(N)
hop.store_patterns(patterns)

# show stored patterns
for i,p in enumerate(patterns):
    plot_pattern(p, shape, f"Stored Pattern {i}", f"lab9_plots/pattern_{i}.png")

# Print the name and roll number
print("Name: Somesh Singh")
print("Roll Number: 233025921")

# Distort a stored pattern (flip some bits)
def distort_pattern(p, flip_fraction=0.2):
    s = p.copy()
    n_flip = int(len(s) * flip_fraction)
    idx = np.random.choice(len(s), size=n_flip, replace=False)
    s[idx] *= -1
    return s, idx

orig = patterns[0]
distorted, flipped_idx = distort_pattern(orig, flip_fraction=0.25)
plot_pattern(distorted, shape, "Distorted Input (25%)", "lab9_plots/distorted_input.png")

# Run asynchronous updates and record energy
final_state, history = hop.asynchronous_update(distorted.copy(), steps=10, random_order=True)
energies = [hop.energy(s) for s in history]

# save intermediate patterns and energy plot
for t,s in enumerate(history):
    plot_pattern(s, shape, f"Step {t}", f"lab9_plots/reconstruct_step_{t}.png")

plt.figure()
plt.plot(energies, marker='o')
plt.title("Energy over asynchronous updates (recovery)")
plt.xlabel("Update step"); plt.ylabel("Energy")
plt.savefig("lab9_plots/energy_recovery.png", dpi=150, bbox_inches='tight')
plt.close()

# print match result
print("Q1/Q2: Distorted -> final recovery match to original?",
      np.array_equal(final_state, orig))
print("Hamming distance from original after recovery:",
      np.sum(final_state != orig))

Name: Somesh Singh
Roll Number: 233025921
Name: Somesh Singh
Roll Number: 233025921
Q1/Q2: Distorted -> final recovery match to original? True
Hamming distance from original after recovery: 0


**Q3. Visualize the energy function and how it evolves over time.**

In [6]:
def test_recall_from_random(hop, patterns, trials=5, steps=20):
    results = []
    for t in range(trials):
        # random initial state
        init = bipolar(np.random.randint(0,2,size=N))
        s, hist = hop.asynchronous_update(init.copy(), steps=steps)
        E = [hop.energy(h) for h in hist]
        results.append((init, hist, E))
    return results

rand_results = test_recall_from_random(hop, patterns, trials=4, steps=15)
# plot energies
plt.figure()
for i,(_,_,E) in enumerate(rand_results):
    plt.plot(E, label=f"trial {i}")
plt.title("Q3: Energy descent from random starts")
plt.xlabel("asynchronous update steps"); plt.ylabel("Energy"); plt.legend()

# Print the name and roll number
print("Name: Somesh Singh")
print("Roll Number: 233025921")

plt.savefig("lab9_plots/energy_random_starts.png", dpi=150, bbox_inches='tight')
plt.close()

Name: Somesh Singh
Roll Number: 233025921


**Q4. Show the use of attractors in memory recall from partial input.**

In [7]:
def partial_input_recall(hop, pattern, mask_fraction=0.5, steps=15):
    s = pattern.copy()
    # mask positions: set to random -1/+1 (unknown)
    n_mask = int(len(s) * mask_fraction)
    mask_idx = np.random.choice(len(s), size=n_mask, replace=False)
    s[mask_idx] = np.random.choice([-1,1], size=n_mask)
    final, history = hop.asynchronous_update(s, steps=steps)
    return s, final, history

# Print the name and roll number
print("Name: Somesh Singh")
print("Roll Number: 233025921")

partial_in, recovered, hist_p = partial_input_recall(hop, patterns[1], mask_fraction=0.4, steps=12)
plot_pattern(partial_in, shape, "Partial input (40% masked)", "lab9_plots/partial_input.png")
plot_pattern(recovered, shape, "Recovered from partial", "lab9_plots/partial_recovered.png")
print("Q4: Partial -> matches stored pattern?", np.array_equal(recovered, patterns[1]))

Name: Somesh Singh
Roll Number: 233025921
Q4: Partial -> matches stored pattern? True


**Q5. Demonstrate failure case: store too many patterns, observe instability.**

In [8]:
def retrieval_accuracy(hop, patterns, flips=0.1, steps=10):
    accuracies = []
    for p in patterns:
        distorted, _ = distort_pattern(p, flip_fraction=flips)
        final, _ = hop.asynchronous_update(distorted.copy(), steps=steps)
        accuracies.append(np.array_equal(final, p))
    return np.mean(accuracies)

max_patterns = int(0.4 * N)  # go up to 40% of N (will exceed capacity)
pattern_list = []
# generate random bipolar patterns
for pcount in range(1, max_patterns+1):
    # add a new random pattern
    newp = bipolar(np.random.randint(0,2,size=N))
    pattern_list.append(newp)

results = []
P_list = list(range(1, max_patterns+1, max(1, max_patterns//20)))
for P in P_list:
    hop.store_patterns(pattern_list[:P])
    acc = retrieval_accuracy(hop, pattern_list[:P], flips=0.1, steps=15)
    results.append((P, acc))
    print(f"Stored P={P} patterns -> retrieval accuracy (after 10% flip): {acc:.3f}")

# Print the name and roll number
print("Name: Somesh Singh")
print("Roll Number: 233025921")

# Plot accuracy vs P
Ps = [r[0] for r in results]
Accs = [r[1] for r in results]
plt.figure()
plt.plot(Ps, Accs, '-o')
plt.axvline(0.14*N, color='red', linestyle='--', label='approx capacity 0.14*N')
plt.xlabel("Number of stored patterns P")
plt.ylabel("Retrieval accuracy")
plt.title("Q5: Retrieval accuracy vs number of stored patterns (capacity)")
plt.legend()
plt.savefig("lab9_plots/retrieval_vs_P.png", dpi=150, bbox_inches='tight')
plt.close()

# Also show an example of spurious state when overloaded
# store many patterns and try to recall one
P_over = int(0.3 * N)
hop.store_patterns(pattern_list[:P_over])
pidx = 0
p = pattern_list[pidx]
distorted, _ = distort_pattern(p, flip_fraction=0.2)
final, hist_over = hop.asynchronous_update(distorted.copy(), steps=30)
plot_pattern(distorted, shape, "Overloaded: Distorted input", "lab9_plots/overload_distorted.png")
plot_pattern(final, shape, "Overloaded: Final (may be spurious)", "lab9_plots/overload_final.png")
print("Q5 example: when overloaded, final equals original?", np.array_equal(final, p))

Stored P=1 patterns -> retrieval accuracy (after 10% flip): 1.000
Stored P=2 patterns -> retrieval accuracy (after 10% flip): 1.000
Stored P=3 patterns -> retrieval accuracy (after 10% flip): 1.000
Stored P=4 patterns -> retrieval accuracy (after 10% flip): 1.000
Stored P=5 patterns -> retrieval accuracy (after 10% flip): 1.000
Stored P=6 patterns -> retrieval accuracy (after 10% flip): 1.000
Stored P=7 patterns -> retrieval accuracy (after 10% flip): 1.000
Stored P=8 patterns -> retrieval accuracy (after 10% flip): 1.000
Stored P=9 patterns -> retrieval accuracy (after 10% flip): 0.778
Stored P=10 patterns -> retrieval accuracy (after 10% flip): 0.400
Stored P=11 patterns -> retrieval accuracy (after 10% flip): 0.545
Stored P=12 patterns -> retrieval accuracy (after 10% flip): 0.500
Stored P=13 patterns -> retrieval accuracy (after 10% flip): 0.308
Stored P=14 patterns -> retrieval accuracy (after 10% flip): 0.286
Stored P=15 patterns -> retrieval accuracy (after 10% flip): 0.333
Stor

# -------------------------
# Summary of saved plots
# -------------------------

In [9]:
print("\nSaved plots (lab9_plots/):")
saved = [
 "pattern_0.png", "pattern_1.png", "pattern_2.png",
 "distorted_input.png",
 "reconstruct_step_0.png", "reconstruct_step_1.png", "reconstruct_step_2.png",
 "reconstruct_step_3.png", "reconstruct_step_4.png", "reconstruct_step_5.png",
 "energy_recovery.png",
 "energy_random_starts.png",
 "partial_input.png", "partial_recovered.png",
 "retrieval_vs_P.png",
 "overload_distorted.png", "overload_final.png"
]
for p in saved:
    print("lab9_plots/" + p)

# Print the name and roll number
print("Name: Somesh Singh")
print("Roll Number: 233025921")


Saved plots (lab9_plots/):
lab9_plots/pattern_0.png
lab9_plots/pattern_1.png
lab9_plots/pattern_2.png
lab9_plots/distorted_input.png
lab9_plots/reconstruct_step_0.png
lab9_plots/reconstruct_step_1.png
lab9_plots/reconstruct_step_2.png
lab9_plots/reconstruct_step_3.png
lab9_plots/reconstruct_step_4.png
lab9_plots/reconstruct_step_5.png
lab9_plots/energy_recovery.png
lab9_plots/energy_random_starts.png
lab9_plots/partial_input.png
lab9_plots/partial_recovered.png
lab9_plots/retrieval_vs_P.png
lab9_plots/overload_distorted.png
lab9_plots/overload_final.png
Name: Somesh Singh
Roll Number: 233025921
